# Text Expanding

Goal of this notebook is practicing and understanding prompts used for text expanding.

Examples sourced from [Deeplearning.ai ChatGPT Prompt Engineering for Developers](https://learn.deeplearning.ai/chatgpt-prompt-eng/lesson/7/expanding) course.

LLM Model: [dolly-v2-3b](https://huggingface.co/databricks/dolly-v2-3b)

In [1]:
!export PYTORCH_ENABLE_MPS_FALLBACK=1
import time

import torch
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# print(torch.backends.mps.is_available())
# print(torch.backends.mps.is_built())
# print(torch.cuda.is_available())

In [2]:
generate_text = pipeline(model="databricks/dolly-v2-3b", torch_dtype=torch.bfloat16,
                         trust_remote_code=True, device_map="auto", return_full_text=True)

simple_prompt = PromptTemplate(
    input_variables=["instruction"],
    template="{instruction}")

prompt_with_context = PromptTemplate(
    input_variables=["instruction", "context"],
    template="{instruction}\n\nInput:\n{context}")

hf_pipeline = HuggingFacePipeline(pipeline=generate_text)

llm_chain = LLMChain(llm=hf_pipeline, prompt=simple_prompt)
llm_context_chain = LLMChain(llm=hf_pipeline, prompt=prompt_with_context)


def get_response(input_prompt, context=None):
    if context is None:
        return llm_chain.predict(instruction=input_prompt).lstrip()
    else:
        return llm_context_chain.predict(instruction=input_prompt, context=context).lstrip()


def reply_prompt(input_prompt, context=None):
    start = time.time()
    response = get_response(input_prompt, context)
    end = time.time()
    duration_seconds = end - start
    print(response)
    print(f"\nProcessing duration: {duration_seconds:0.2f} seconds")


## Challenge: Customize automatic reply of complaint

In [3]:
# given the sentiment from the lesson on "inferring",
# and the original customer message, customize the email
sentiment = "negative"

# review for a blender
review = f"""
So, they still had the 17 piece system on seasonal sale for around $49 in the month of November, about half off, but for some reason (call it price gouging) around the second week of December the prices all went up to about anywhere from between $70-$89 for the same system. And the 11 piece system went up around $10 or so in price also from the earlier sale price of $29. So it looks okay, but if you look at the base, the part where the blade locks into place doesn’t look as good as in previous editions from a few years ago, but I plan to be very gentle with it (example, I crush very hard items like beans, ice, rice, etc. in the blender first then pulverize them in the serving size I want in the blender then switch to the whipping blade for a finer flour, and use the cross cutting blade first when making smoothies, then use the flat blade if I need them finer/less pulpy). Special tip when making smoothies, finely cut and freeze the fruits and vegetables (if using spinach-lightly stew soften the spinach then freeze until ready for use-and if making sorbet, use a small to medium sized food processor) that you plan to use that way you can avoid adding so much ice if at all-when making your smoothie. After about a year, the motor was making a funny noise. I called customer service but the warranty expired already, so I had to buy another one. FYI: The overall quality has gone done in these types of products, so they are kind of counting on brand recognition and consumer loyalty to maintain sales. Got it in about two days.
"""

In [4]:
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for their review.
If the sentiment is negative, apologize and suggest that they can reach out to customer service.
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""
reply_prompt(prompt)

Apologize, suggest that the customer service team can help them.

Processing duration: 317.46 seconds


In [5]:
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for their review.
If the sentiment is negative, apologize and suggest that they can reach out to customer service.
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""
reply_prompt(prompt)

The sentiment of the customer review is negative.
Apologize and suggest that they can reach out to customer service.
Include specific details from the review.

Processing duration: 329.64 seconds
